In [17]:
import numpy as np
import pandas as pd
import random
import os

In [2]:
# 删除KPI_new的第一周数据
# path = '../data/KPI_new.csv'
# o = open(path, 'rb')
# df = pd.read_csv(o)
# df = df[2016:]
# df.info()
# # csvFile = open('../data/KPI_week2.csv', 'w')
# df.to_csv( '../data/KPI_week2.csv', index=None)

In [3]:
# 取出5种属性的所有属性值
def find_total_attrs(path):
	attrs = os.listdir(path)
	total_attrs = []
	for attr in attrs:
		vals = os.listdir(path + attr)
		vals= [ x.split('=')[1].split('.')[0] for x in vals ]
		if 'unknown' in vals:
			vals.remove('unknown')
		total_attrs.append(vals)
	# print(total_attrs)
	return total_attrs


# 一共5层，每层1~5种root cause,共25种
def find_root_causes(total_attrs):
	layers, kinds = 5, 5
	root_causes = []
	for i in range(layers):
		layer_vals = []
		for j in range(kinds):
			vals = []
			# 从5个属性里随机选择 i 个(第一层1个，第二层2个...)
			random_nums = random.sample([x for x in range(5)], i + 1)	
			for k in range(i + 1):
				vals.append( random.sample(total_attrs[random_nums[k]], 1)[0] )	
			layer_vals.append(vals)
			temp = layer_vals[:]		# 深拷贝
			root_causes.append(temp)
	return root_causes


## 随机找出25种根因

In [4]:
total_attrs = find_total_attrs('../data/result/')
root_causes = find_root_causes(total_attrs)
np.save('../data/root_causes.npy', root_causes)

for root_cause in root_causes:
	print(root_cause)

[['l2']]
[['l2'], ['i26']]
[['l2'], ['i26'], ['i65']]
[['l2'], ['i26'], ['i65'], ['e05']]
[['l2'], ['i26'], ['i65'], ['e05'], ['l1']]
[['i119', 'p18']]
[['i119', 'p18'], ['c6', 'l4']]
[['i119', 'p18'], ['c6', 'l4'], ['i101', 'c7']]
[['i119', 'p18'], ['c6', 'l4'], ['i101', 'c7'], ['l3', 'e11']]
[['i119', 'p18'], ['c6', 'l4'], ['i101', 'c7'], ['l3', 'e11'], ['e01', 'p12']]
[['i79', 'l4', 'e11']]
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62']]
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62'], ['p19', 'c1', 'i58']]
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62'], ['p19', 'c1', 'i58'], ['i31', 'e04', 'c7']]
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62'], ['p19', 'c1', 'i58'], ['i31', 'e04', 'c7'], ['c7', 'l3', 'p33']]
[['c6', 'l3', 'e04', 'p26']]
[['c6', 'l3', 'e04', 'p26'], ['c3', 'p24', 'i61', 'l1']]
[['c6', 'l3', 'e04', 'p26'], ['c3', 'p24', 'i61', 'l1'], ['p07', 'l2', 'c8', 'e07']]
[['c6', 'l3', 'e04', 'p26'], ['c3', 'p24', 'i61', 'l1'], ['p07', 'l2', 'c8', 'e07'], ['i70', 'p12', 'c1', 'e12']]
[['c6', 'l3

## 随机找出25个timestamp注入异常

In [5]:
# 计算异常
def compute_anomaly(path):
    o = open(path, 'rb')
    df = pd.read_csv(o)
    length = len(df)

    # 随机选取一个时间戳(为方便计算不选头尾处时间戳)
    rand_idx = random.randint(150, length - 150)
    timestamp = df.loc[rand_idx][0]						# 随机选择的timestamp
    temp_df = df[rand_idx - 144 : rand_idx + 144]		# 时间间隔为5分钟，共14天数据，选取一天的数据
    kpi = temp_df['KPI'].as_matrix()
    kpi = [float(i) for i in kpi]
    # print(kpi)
    mean = np.mean(kpi)
    std = np.std(kpi)
    total_anomaly = int(mean + 5*std)					# 此timestamp上的总KPI异常
    # print(total_anomaly)
    return timestamp, total_anomaly


# 分发异常
def distribute_anomaly(root_cause, path, root_cause_dict):
    timestamp, total_anomaly = compute_anomaly(path)
#     print(root_cause, timestamp)
    root_cause_dict[timestamp] = root_cause
    path2 = '../data/origin/' + str(timestamp) + '.csv'
    path3 = '../data/anomaly/' + str(timestamp) + '.csv'
    locs = []           # 后代叶子元素的索引
    o = open(path2, 'rb')
    df = pd.read_csv(o, names  = ['i', 'e', 'c', 'p', 'l', 'kpi'])
    for x in root_cause:
        for idx in df.index:
            i, e, c, p, l = df.loc[idx]['i'], df.loc[idx]['e'], df.loc[idx]['c'], df.loc[idx]['p'], df.loc[idx]['l']
            if i in x or e in x or c in x or p in x or l in x:
                locs.append(idx)

    # 若此时间戳不含此属性值，则重新找一个时间戳
    if len(locs) == 0:
        distribute_anomaly(root_cause, path, root_cause_dict)
        return

#     print(locs)
    anomaly = int(total_anomaly / len(locs))
    df_out = pd.DataFrame(columns = ['i', 'e', 'c', 'p', 'l', 'kpi'])
    for idx in locs:
        i, e, c, p, l = df.loc[idx]['i'], df.loc[idx]['e'], df.loc[idx]['c'], df.loc[idx]['p'], df.loc[idx]['l']
#                 print(i, e, c, p, l)
        df_out.loc[0] = {'i':i, 'e':e, 'c':c, 'p':p, 'l':l, 'kpi':df.loc[idx]['kpi'] + anomaly}
        df = pd.concat([df[:idx], df_out, df[idx+1:]], ignore_index=True)
    csvfile = open(path3, 'w')
    df.to_csv(path3, header=None, index=None)

In [6]:
# 使用25种根因随机选择timestamp注入异常
path = '../data/KPI_week2.csv'
root_cause_dict = {}
for root_cause in root_causes:
    distribute_anomaly(root_cause, path, root_cause_dict)

[['l2']] 1536835800000
[['l2'], ['i26']] 1536666300000
[['l2'], ['i26'], ['i65']] 1536823500000
[['l2'], ['i26'], ['i65'], ['e05']] 1536501600000
[['l2'], ['i26'], ['i65'], ['e05'], ['l1']] 1536794100000
[['i119', 'p18']] 1536624900000
[['i119', 'p18'], ['c6', 'l4']] 1536592200000
[['i119', 'p18'], ['c6', 'l4'], ['i101', 'c7']] 1536749100000
[['i119', 'p18'], ['c6', 'l4'], ['i101', 'c7'], ['l3', 'e11']] 1536716700000
[['i119', 'p18'], ['c6', 'l4'], ['i101', 'c7'], ['l3', 'e11'], ['e01', 'p12']] 1536435000000
[['i79', 'l4', 'e11']] 1536793200000
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62']] 1536670500000
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62'], ['p19', 'c1', 'i58']] 1536617100000
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62'], ['p19', 'c1', 'i58'], ['i31', 'e04', 'c7']] 1536505800000
[['i79', 'l4', 'e11'], ['l4', 'c5', 'i62'], ['p19', 'c1', 'i58'], ['i31', 'e04', 'c7'], ['c7', 'l3', 'p33']] 1536507900000
[['c6', 'l3', 'e04', 'p26']] 1536423000000
[['c6', 'l3', 'e04', 'p26'], ['c3', 'p24', 

In [21]:
# 将注入异常的25个时间戳保存成csv文件
timestamps = list( root_cause_dict.keys() )
df_timestamps = pd.DataFrame(timestamps, index=None)
csvFile = open('../data/abnormal_timestamps.csv', 'w')
df_timestamps.to_csv('../data/abnormal_timestamps.csv', header=None, index=None)

# 将每个时间戳对应的异常根因保存成文件
new_path = '../data/timestamp_root_cause/'
if not os.path.exists(new_path):
    os.makedirs(new_path)
for (k, v) in root_cause_dict.items():
    path = new_path + str(k) + '.csv'
    csvFile = open(path, 'w')
    df = pd.DataFrame(v)
    df.to_csv(path,header=None, index=None)